<a href="https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/02_bullet_bonds.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Bullet Bonds

## Learning Objectives

- Define the cash flows of a bullet bond. 
- Understand how the price of a bond is quoted.
- Learn about duration and convexity of a bullet bond. 



## Cash Flow Structure

A bullet bond returns the principal at maturity. This is an example of a bullet bond:

In [8]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import pandas as pd
from IPython.display import display

# Example bond
bond = FixedRateBullet(
    "2020-01-01", "2030-01-01", 5, 1, notional=100, day_count_convention="30/365"
)



df = pd.DataFrame(bond.cash_flows())
df.columns = ["Value"]
display(df.style.hide(axis="index"))

Value
5.000000
5.000000
5.000000
5.000000
5.000000
5.000000
5.000000
5.000000
5.000000
105.000000


## Calculating Bond Price from Discount Curve

In order to price a bond correctly, we just need to compute the present value of the future cash flows. The cash flows consist of the periodic coupon payments and the face value paid at maturity, in the case of the bullet bond. If the bond amortizes before maturity, these amortization payments must also be included in the bond cash flows.

## Discount Rate

In order to discount the cash flows we need to find the discount rate, which is found using the exponential function: $P = F \times e^{-rt}$.

In [9]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

bond_discount_example = FixedRateBullet(
    issue_dt="2026-01-01",
    maturity="2031-01-01",
    cpn=5,
    cpn_freq=1,
    notional=100,
    yield_to_maturity=0.05,
    settlement_date="2027-01-01",
)

price = bond_discount_example.get_price()
print(f"Bond Price = {price:.2f}")

Bond Price = 99.78


## Duration

Duration measures a bond's price sensitivity to changes in interest rates and represents the weighted average time to receive the bond's cash flows. Mathematically, duration approximates the percentage change in a bond's price for a small change in yield, expressed as:

$$D = -\frac{1}{P} \cdot \frac{dP}{dr}$$

The negative sign reflects the inverse relationship between bond prices and yields.

In [10]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

bond_duration_example = FixedRateBullet(
    issue_dt="2020-01-01",
    maturity="2025-01-01",
    cpn=5,
    cpn_freq=1,
    notional=1000,
    yield_to_maturity=5,
    settlement_date="2021-01-01",
)

### Macaulay Duration

Macaulay duration is the weighted average time until a bond's cash flows are received, expressed in years. It is calculated as:

$$\frac{\sum_{t=1}^n \frac{t \cdot C_t}{(1 + y)^t} + \frac{n \cdot FV}{(1 + y)^n}}{P}$$

where:
- $C_t$: Cash flow (coupon payment) at time $t$
- $FV$: Face value of the bond
- $y$: Yield to maturity (per period)
- $n$: Number of periods until maturity
- $P$: Present value (current price) of the bond

For a zero-coupon bond, since there are no coupon payments ($C_t = 0$), the Macaulay duration simplifies to the time to maturity, $T - t$, where $T$ is the maturity date and $t$ is the current date.

In [11]:
macaulay_duration = bond_duration_example.macaulay_duration()
print(macaulay_duration)

1.1175133907


### Modified Duration

Modified duration directly measures the percentage change in a bond's price for a 1% change in yield. It is derived from the Macaulay duration as:

$$D^{\text{Mod}} = \frac{D^{\text{MC}}}{1 + \frac{y}{k}}$$

where:
- $y$: Annual yield to maturity
- $k$: Number of compounding periods per year (e.g., $k = 2$ for semi-annual, $k = 1$ for annual)

For a zero-coupon bond with annual compounding ($k = 1$), the modified duration is:

$$D^{\text{Mod}} = \frac{T - t}{1 + y}$$


In [12]:
modified_duration = bond_duration_example.modified_duration()

print(modified_duration)

0.3192895402



## Convexity

Convexity measures the sensitivity of a bond’s duration to changes in interest rates and captures the non-linear relationship between bond prices and yields. It is the second derivative of the bond’s price with respect to the yield, scaled by the price:

$$C = \frac{1}{P} \cdot \frac{d^2P}{dr^2}$$

For a zero-coupon bond with annual compounding, the convexity is:

$$C = \frac{(T - t)(T - t + 1)}{(1 + y)^2}$$

In [13]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

convexity_example_bond = FixedRateBullet(
    issue_dt="2020-01-01",
    maturity="2025-01-01",
    cpn=5,
    cpn_freq=1,
    notional=1000,
    yield_to_maturity=5,
    settlement_date="2021-01-01",
)

convexity = convexity_example_bond.convexity()

print(convexity)

0.162101523
